In [76]:
import torch
import numpy as np
from PIL import Image

from clip import clip

In [77]:
model, preprocess_img = clip.load('ViT-B/32')

In [78]:
model

RecursiveScriptModule(
  original_name=Multimodal
  (visual): RecursiveScriptModule(
    original_name=VisualTransformer
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (ln_pre): RecursiveScriptModule(original_name=LayerNorm)
    (transformer): RecursiveScriptModule(
      original_name=Transformer
      (resblocks): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=ResidualAttentionBlock
          (attn): RecursiveScriptModule(
            original_name=MultiheadAttention
            (out_proj): RecursiveScriptModule(original_name=_LinearWithBias)
          )
          (ln_1): RecursiveScriptModule(original_name=LayerNorm)
          (mlp): RecursiveScriptModule(
            original_name=Sequential
            (c_fc): RecursiveScriptModule(original_name=Linear)
            (gelu): RecursiveScriptModule(original_name=QuickGELU)
            (c_proj): RecursiveScriptModule(original_name=Linear)
          )
   

In [81]:
with Image.open('./n07920052_espresso.jpeg') as f:
    espresso_img = f.convert('RGB')

In [82]:
image = preprocess_img(espresso_img)

In [46]:
image_proc = torch.tensor(np.stack([image])).cuda()

In [47]:
image_proc

tensor([[[[ 1.6968,  1.6822,  1.6530,  ..., -0.5368, -0.5514, -0.5514],
          [ 1.7114,  1.7114,  1.6822,  ..., -0.5368, -0.5514, -0.5660],
          [ 1.6822,  1.6822,  1.6530,  ..., -0.5368, -0.5514, -0.5660],
          ...,
          [ 1.0982,  1.3610,  1.4924,  ...,  0.6457,  0.6603,  0.6603],
          [ 1.0982,  1.2004,  1.2588,  ...,  0.6749,  0.6749,  0.6749],
          [ 1.0544,  1.0690,  1.1274,  ...,  0.6749,  0.6749,  0.6603]],

         [[ 1.7297,  1.6847,  1.6547,  ..., -0.7466, -0.7616, -0.7616],
          [ 1.7447,  1.7297,  1.7147,  ..., -0.7466, -0.7616, -0.7616],
          [ 1.7147,  1.7297,  1.6997,  ..., -0.7466, -0.7616, -0.7766],
          ...,
          [ 1.3245,  1.5946,  1.7147,  ...,  0.9043,  0.9193,  0.9193],
          [ 1.2795,  1.3845,  1.4145,  ...,  0.9343,  0.9343,  0.9343],
          [ 1.2044,  1.2044,  1.2795,  ...,  0.9343,  0.9343,  0.9193]],

         [[ 1.4633,  1.3922,  1.3354,  ..., -1.1389, -1.1532, -1.1532],
          [ 1.4918,  1.4491,  

In [53]:
some_string = 'an image of espresso'

In [60]:
some_tokens = clip.tokenize(some_string).cuda()
# tokens_proc = torch.tensor(np.stack([some_tokens])).cuda()

In [61]:
with torch.no_grad():
    image_features = model.encode_image(image_proc).float()
    text_features = model.encode_text(some_tokens).float()

In [63]:
text_probs

tensor([[1.]], device='cuda:0')

In [65]:
strings_test = ["a diagram", "a dog", "a cat", 'a picture of espresso']

In [83]:
image = preprocess_img(espresso_img).unsqueeze(0).cuda()
text = clip.tokenize(strings_test).cuda()

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)

In [87]:
probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[4.99e-05 7.15e-07 1.01e-06 1.00e+00]]


In [85]:
logits_per_image

tensor([[18.6875, 14.4297, 14.8047, 28.5938]], device='cuda:0',
       dtype=torch.float16)

In [86]:
logits_per_text

tensor([[18.6875],
        [14.4297],
        [14.8047],
        [28.5938]], device='cuda:0', dtype=torch.float16)